<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建立索引清單

In [1]:
import json
import csv
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import tqdm

In [2]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage3/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

ValueError: ignored

In [ ]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/similarity_list_stage_2.json'
# response = urlopen(url)
# similarity_list = json.loads(response.read())

In [ ]:
# similarity_list

In [ ]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/column_filter.json'
# response = urlopen(url)
# column_filter = json.loads(response.read())

In [ ]:
def Union_calculation(input):
  union = set()
  for values in list(input.values()):
    if len(union)==0:
      union = set(values)
    else:
      union = union & set(values)
  return union

In [ ]:
similarity_list = {}
column_filter={}
for key,labels in data.items():
  row_1 = pd.DataFrame(labels).T
  mask = row_1 > 0
  filter = row_1[mask].dropna(axis=1).columns.tolist()
  column_filter[key]=filter
  filter_list = {}
  for i in filter:
    filter_list[i] = df.iloc[np.flatnonzero(df[i])].index.tolist()
  similarity_list[key] = list(set(filter_list)) #list(Union_calculation(filter_list))

In [ ]:
print(list(similarity_list.keys())[0])
similarity_list[list(similarity_list.keys())[0]]

In [ ]:
# similarity_list

In [ ]:
index=3
object_list = df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]]
object_list.iloc[:].tolist()

In [ ]:
df[column_filter[list(column_filter.keys())[index]]]

In [ ]:
# index = 100
index = list(column_filter.keys()).index('1033')
print("Object:",list(column_filter.keys())[index])
print("index:",column_filter[list(column_filter.keys())[index]])
print("values:",df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist())
print("same_index_rows:",similarity_list[list(similarity_list.keys())[index]])
index_filter = df[column_filter[list(column_filter.keys())[index]]]#.loc[similarity_list[list(similarity_list.keys())[index]]]
index_filter

In [ ]:
pd.DataFrame(df[column_filter[list(column_filter.keys())[index]]].loc[['1073','741']])

In [ ]:
filter_dict={}
for key ,value in zip(index_filter.index,index_filter.iloc[:,:].values):
    if str(value).count('0')/len(value) <= 0.5 and key != index:
        filter_dict[key] = list(value)
    # break

In [ ]:
pd.DataFrame(df[column_filter[list(column_filter.keys())[index]]].loc[list(filter_dict.keys())])

# 進行迭代分析

In [ ]:
df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist()

In [ ]:
results = {}
for i in list(column_filter.keys()):
  index_object = column_filter[i]
  # print("Object:",i,index_object)
  compare_object = df[column_filter[i]].loc[i].tolist()
  results[i]={"key":index_object}
  compare_table = df[column_filter[i]].loc[similarity_list[i]]
  temp={}
  for key, value in zip(compare_table.iloc[:,:].index,compare_table.iloc[:,:].values):
    if key!=i:temp[key] = value.tolist()
  if len(temp)>0:
    results[i]["values"]=temp
  else:
    del results[i]
  # print('----------------------')
  # break
  

In [ ]:
# compare_table.iloc[:,:].index

In [ ]:
# results

# 文章篩選

In [ ]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage1/dataTrainComplete.json'
response = urlopen(url)
origin_assay = json.loads(response.read())

In [ ]:
origin_assay['1033']

In [ ]:
origin_assay['415']

# 匯出檔案

##儲存為JSON格式

In [ ]:
import json
with open('./results.json', 'w', encoding='utf8') as f:
    json.dump(results, f, ensure_ascii=False)

## 從Colab下載檔案到電腦

In [ ]:
from google.colab import files
files.download("/content/results.json")